# Super Bowl Ticket Prices: 1967 to Today
### An Analysis of 60 Years of the Biggest Game in Sports

This notebook analyzes the evolution of Super Bowl ticket prices from the first AFL-NFL World Championship Game in 1967 through Super Bowl LX in 2026. We examine nominal prices, inflation-adjusted costs, the emergence of the secondary (resale) market, and estimated price distributions across seating tiers.

**Data sources:** Casino.org, TickPick, GOBankingRates, Yahoo Sports, Marco News/USA TODAY, BLS CPI-U  
**Last updated:** February 2026

In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.io as pio
from scipy import stats
from pathlib import Path

# Configuration
pio.templates.default = "plotly_white"
EXPORT_DIR = Path("../exports")
EXPORT_DIR.mkdir(exist_ok=True)
DATA_DIR = Path("../data")

# Colorblind-safe palette
COLORS = {
    'nominal': '#636EFA',       # Blue
    'adjusted': '#EF553B',      # Red-orange
    'face_value': '#00CC96',    # Teal
    'resale': '#AB63FA',        # Purple
    'cpi': '#FFA15A',           # Orange
    'histogram': '#19D3F3',     # Cyan
    'annotation': '#FF6692',    # Pink
}

TARGET_YEAR = 2025  # Adjust all prices to this year's dollars

# Quick kaleido sanity check
test_fig = go.Figure(data=go.Scatter(x=[1], y=[1]))
test_fig.write_image(EXPORT_DIR / "_test.png")
(EXPORT_DIR / "_test.png").unlink()
print("Kaleido PNG export: OK")

E:\Python\Python38-32\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
E:\Python\Python38-32\lib\site-packages\numpy\.libs\libopenblas.D6ALFJ4QQDWP6YNOQJNPYL27LRE6SILT.gfortran-win32.dll
E:\Python\Python38-32\lib\site-packages\numpy\.libs\libopenblas_v0.3.21-gcc_8_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
<ipython-input-1-a416acfc04d9>:31: DeprecationWarning:


Support for Kaleido versions less than 1.0.0 is deprecated and will be removed after September 2025.
Please upgrade Kaleido to version 1.0.0 or greater (`pip install 'kaleido>=1.0.0'` or `pip install 'plotly[kaleido]'`).




Kaleido PNG export: OK


## Why Adjust for Inflation?

A $12 ticket in 1967 might sound cheap, but what could $12 buy back then? Comparing prices across decades in raw (nominal) dollars is misleading because the purchasing power of a dollar changes over time.

We use the **Consumer Price Index (CPI-U)** published by the Bureau of Labor Statistics to convert all prices to constant 2025 dollars. The CPI measures the average change over time in the prices paid by urban consumers for a basket of goods and services.

**The formula:**
```
adjusted_price = nominal_price × (CPI_target_year / CPI_original_year)
```

**Example:** SB I in 1967 cost $12. CPI in 1967 was 33.4; in 2025 it's 321.9.  
$12 × (321.9 / 33.4) = **$115.63 in 2025 dollars**.

> **Tip for your own projects:** Always disclose the CPI base year and adjustment target year. Writing "in 2025 dollars" is more transparent than just saying "inflation-adjusted." Also note that CPI represents *general* inflation — specific categories (housing, healthcare, entertainment) may inflate at different rates.

In [2]:
# Load data
prices = pd.read_csv(DATA_DIR / "superbowl_ticket_prices.csv")
cpi = pd.read_csv(DATA_DIR / "cpi_annual_averages.csv")

print(f"Loaded {len(prices)} Super Bowl records ({prices['year'].min()}-{prices['year'].max()})")
print(f"Loaded CPI data for {len(cpi)} years ({cpi['year'].min()}-{cpi['year'].max()})")
prices.head()

Loaded 60 Super Bowl records (1967-2026)
Loaded CPI data for 60 years (1967-2026)


,sb_number,sb_roman,year,date,city,stadium,winner,loser,score,face_value_low,face_value_high,avg_ticket_price,resale_avg,attendance,source_notes
0,1,I,1967,1967-01-15,Los Angeles,Los Angeles Memorial Coliseum,Green Bay Packers,Kansas City Chiefs,35-10,6.0,12,12,NaN,61946,First AFL-NFL Championship Game; tickets as lo...
1,2,II,1968,1968-01-14,Miami,Orange Bowl,Green Bay Packers,Oakland Raiders,33-14,6.0,12,12,NaN,75546,NaN
2,3,III,1969,1969-01-12,Miami,Orange Bowl,New York Jets,Baltimore Colts,16-7,6.0,12,12,NaN,75389,Joe Namath guarantee
3,4,IV,1970,1970-01-11,New Orleans,Tulane Stadium,Kansas City Chiefs,Minnesota Vikings,23-7,7.5,15,15,NaN,80562,First price increase
4,5,V,1971,1971-01-17,Miami,Orange Bowl,Baltimore Colts,Dallas Cowboys,16-13,7.5,15,15,NaN,79204,NaN


In [3]:
# Merge and compute inflation-adjusted prices
df = prices.merge(cpi, on='year', how='left')

cpi_target = cpi.loc[cpi['year'] == TARGET_YEAR, 'cpi_annual_avg'].values[0]
print(f"Target CPI ({TARGET_YEAR}): {cpi_target}")

for col in ['face_value_low', 'face_value_high', 'avg_ticket_price', 'resale_avg']:
    adj_col = f'{col}_adj'
    df[adj_col] = df[col] * (cpi_target / df['cpi_annual_avg'])

# Computed face value midpoint (consistent across all years)
df['face_value_mid'] = (df['face_value_low'] + df['face_value_high']) / 2
df['face_value_mid_adj'] = df['face_value_mid'] * (cpi_target / df['cpi_annual_avg'])

# Era and decade columns
df['decade'] = (df['year'] // 10) * 10
df['era'] = pd.cut(df['year'],
    bins=[1966, 1979, 1989, 1999, 2009, 2019, 2030],
    labels=['1967-1979', '1980-1989', '1990-1999', '2000-2009', '2010-2019', '2020-2026'])

# Year-over-year changes
df['yoy_pct_change'] = df['avg_ticket_price'].pct_change() * 100
df['cpi_yoy_pct_change'] = df['cpi_annual_avg'].pct_change() * 100

# Sanity check: SB I adjusted price
sb1_adj = df.loc[df['sb_number'] == 1, 'avg_ticket_price_adj'].values[0]
print(f"\nSanity check: SB I $12 in 1967 = ${sb1_adj:.2f} in {TARGET_YEAR} dollars")
assert abs(sb1_adj - 115.63) < 1.0, f"Expected ~$115.63, got ${sb1_adj:.2f}"
print("Inflation math verified.")

df.head()

Target CPI (2025): 321.9

Sanity check: SB I $12 in 1967 = $115.65 in 2025 dollars
Inflation math verified.


,sb_number,sb_roman,year,date,city,stadium,winner,loser,score,face_value_low,...,face_value_low_adj,face_value_high_adj,avg_ticket_price_adj,resale_avg_adj,face_value_mid,face_value_mid_adj,decade,era,yoy_pct_change,cpi_yoy_pct_change
0,1,I,1967,1967-01-15,Los Angeles,Los Angeles Memorial Coliseum,Green Bay Packers,Kansas City Chiefs,35-10,6.0,...,57.826347,115.652695,115.652695,NaN,9.00,86.739521,1960,1967-1979,NaN,NaN
1,2,II,1968,1968-01-14,Miami,Orange Bowl,Green Bay Packers,Oakland Raiders,33-14,6.0,...,55.500000,111.000000,111.000000,NaN,9.00,83.250000,1960,1967-1979,0.0,4.191617
2,3,III,1969,1969-01-12,Miami,Orange Bowl,New York Jets,Baltimore Colts,16-7,6.0,...,52.626703,105.253406,105.253406,NaN,9.00,78.940054,1960,1967-1979,0.0,5.459770
3,4,IV,1970,1970-01-11,New Orleans,Tulane Stadium,Kansas City Chiefs,Minnesota Vikings,23-7,7.5,...,62.222938,124.445876,124.445876,NaN,11.25,93.334407,1970,1967-1979,25.0,5.722071
4,5,V,1971,1971-01-17,Miami,Orange Bowl,Baltimore Colts,Dallas Cowboys,16-13,7.5,...,59.611111,119.222222,119.222222,NaN,11.25,89.416667,1970,1967-1979,0.0,4.381443


In [4]:
# Data quality check
print("Missing values:")
print(df[['face_value_low', 'face_value_high', 'avg_ticket_price', 'resale_avg', 'cpi_annual_avg']].isnull().sum())
print(f"\nYears with resale data: {df['resale_avg'].notna().sum()} of {len(df)}")
print(f"\nNominal avg price range: ${df['avg_ticket_price'].min():,.0f} - ${df['avg_ticket_price'].max():,.0f}")
print(f"Adjusted avg price range: ${df['avg_ticket_price_adj'].min():,.0f} - ${df['avg_ticket_price_adj'].max():,.0f}")

# Verify no NaN in CPI merge
assert df['cpi_annual_avg'].notna().all(), "CPI merge has NaN values!"
print("\nCPI merge: all years matched.")

Missing values:
face_value_low       0
face_value_high      0
avg_ticket_price     0
resale_avg          40
cpi_annual_avg       0
dtype: int64

Years with resale data: 20 of 60

Nominal avg price range: $12 - $8,907
Adjusted avg price range: $98 - $9,410

CPI merge: all years matched.


## Data Quality Notes

1. **Face values before ~1990** were single-tier (one price for all seats). The `face_value_low` for these years is estimated at ~50% of the reported price based on sparse sourcing. Multi-tier pricing started in the late 1990s.
2. **Resale/secondary market data** is only reliably available from ~2007 onward, when platforms like StubHub began tracking prices systematically.
3. **Sources sometimes disagree** by $25-$100 on face values for the same year. We cross-referenced 4+ sources and documented conflicts in `data/sources.md`.
4. **The `avg_ticket_price` column** uses the most commonly cited "average" from published sources. For early years this equals face value; for recent years it may reflect blended face/market figures.

> **Tip:** In your own analyses, always document where numbers disagree between sources and how you resolved conflicts. Transparency about data limitations builds credibility with readers.

---
## Chart 1: The Big Picture — Nominal vs. Inflation-Adjusted Prices

In [5]:
fig = go.Figure()

# Nominal price line
fig.add_trace(go.Scatter(
    x=df['year'], y=df['avg_ticket_price'],
    mode='lines+markers',
    name='Nominal Price',
    line=dict(color=COLORS['nominal'], width=2),
    marker=dict(size=4),
    hovertemplate='%{x}<br>Nominal: $%{y:,.0f}<extra></extra>'
))

# Inflation-adjusted line
fig.add_trace(go.Scatter(
    x=df['year'], y=df['avg_ticket_price_adj'],
    mode='lines+markers',
    name=f'In {TARGET_YEAR} Dollars',
    line=dict(color=COLORS['adjusted'], width=2.5, dash='dot'),
    marker=dict(size=4),
    hovertemplate='%{x}<br>Adjusted: $%{y:,.0f}<extra></extra>'
))

# Key event annotations
annotations = [
    dict(x=1967, y=df.loc[df['year']==1967, 'avg_ticket_price_adj'].values[0],
         text="SB I: face value 12,<br>~116 in 2025 dollars", showarrow=True,
         arrowhead=2, ax=80, ay=-40, font=dict(size=9)),
    dict(x=1986, y=df.loc[df['year']==1986, 'avg_ticket_price_adj'].values[0],
         text="'85 Bears", showarrow=True,
         arrowhead=2, ax=30, ay=-30, font=dict(size=9)),
    dict(x=2002, y=df.loc[df['year']==2002, 'avg_ticket_price_adj'].values[0],
         text="Brady era begins", showarrow=True,
         arrowhead=2, ax=40, ay=-30, font=dict(size=9)),
    dict(x=2021, y=df.loc[df['year']==2021, 'avg_ticket_price_adj'].values[0],
         text="COVID: 25K cap", showarrow=True,
         arrowhead=2, ax=-60, ay=-30, font=dict(size=9)),
    dict(x=2024, y=df.loc[df['year']==2024, 'avg_ticket_price_adj'].values[0],
         text="First Vegas SB", showarrow=True,
         arrowhead=2, ax=40, ay=30, font=dict(size=9)),
]

fig.update_layout(
    title=dict(text='Super Bowl Average Ticket Prices: 1967-2026', font_size=20),
    xaxis_title='Year',
    yaxis_title='Price (USD)',
    yaxis_tickformat='$,.0f',
    legend=dict(x=0.02, y=0.98),
    hovermode='x unified',
    annotations=annotations,
    width=1000, height=550,
    margin=dict(r=40),
)

fig.show()
fig.write_image(EXPORT_DIR / "trend_nominal_vs_adjusted.png", scale=2)

<ipython-input-5-978e2f277aad>:55: DeprecationWarning:


Support for Kaleido versions less than 1.0.0 is deprecated and will be removed after September 2025.
Please upgrade Kaleido to version 1.0.0 or greater (`pip install 'kaleido>=1.0.0'` or `pip install 'plotly[kaleido]'`).




### Design Tips: Trend Lines

- **Dual-axis charts are almost always a bad idea.** Both series here share the same unit (USD), so a single y-axis works naturally. When units differ (e.g., price vs. attendance), consider separate charts instead of a confusing dual-axis.
- **Annotations beat legends** for calling out specific data points. The reader's eye goes to the annotation text right where the data is — no back-and-forth with a legend.
- **The dotted line** for the adjusted series helps distinguish it even in black-and-white prints or for colorblind readers.
- **`hovermode='x unified'`** in Plotly shows all series values at once when hovering — much more useful than individual point hovers.

---
## Chart 2: Face Value vs. Resale Market — The Premium

In [6]:
recent = df[df['resale_avg'].notna()].copy()

fig = go.Figure()

fig.add_trace(go.Bar(
    x=recent['year'],
    y=recent['face_value_mid'],
    name='Face Value (midpoint)',
    marker_color=COLORS['face_value'],
    hovertemplate='%{x}<br>Face value: $%{y:,.0f}<extra></extra>',
))

fig.add_trace(go.Bar(
    x=recent['year'],
    y=recent['resale_avg'],
    name='Resale Market (avg)',
    marker_color=COLORS['resale'],
    hovertemplate='%{x}<br>Resale: $%{y:,.0f}<extra></extra>',
))

# Markup percentage annotations
for _, row in recent.iterrows():
    if row['face_value_mid'] > 0:
        markup = ((row['resale_avg'] - row['face_value_mid']) / row['face_value_mid']) * 100
        fig.add_annotation(
            x=row['year'], y=row['resale_avg'] + 300,
            text=f'+{markup:.0f}%',
            showarrow=False, font=dict(size=9, color='gray')
        )

fig.update_layout(
    title=dict(text='Face Value vs. Secondary Market: The Resale Premium', font_size=18),
    barmode='group',
    xaxis_title='Year',
    yaxis_title='Price (USD)',
    yaxis_tickformat='$,.0f',
    width=1000, height=500,
    bargap=0.15,
    bargroupgap=0.1,
)

fig.show()
fig.write_image(EXPORT_DIR / "face_vs_resale.png", scale=2)

<ipython-input-6-8e753c9a149e>:43: DeprecationWarning:


Support for Kaleido versions less than 1.0.0 is deprecated and will be removed after September 2025.
Please upgrade Kaleido to version 1.0.0 or greater (`pip install 'kaleido>=1.0.0'` or `pip install 'plotly[kaleido]'`).




---
## Chart 3a: Estimated Price Distributions by Year

### Why Simulate Distributions?

Historical Super Bowl ticket data gives us only 1-3 price points per year (low, average, high). To visualize how ticket prices might be *distributed* across seating tiers, we simulate using a **log-normal distribution**.

Why log-normal?
1. **Floor above zero** — tickets always cost something
2. **Right-skewed** — many cheaper seats, few premium seats at the high end
3. **No hard ceiling** — VIP and resale prices can reach extreme highs

We fit the distribution using the reported low, average, and high values, then generate 5,000 simulated price points.

> **Caveat:** These are **estimates**, not observed transaction data. They illustrate the *likely shape* of the price distribution, not exact values. Always label simulated data clearly in any publication.

In [7]:
def simulate_tier_distribution(low, high, avg, n=5000, seed=42):
    """
    Simulate a log-normal price distribution given reported low, avg, high.
    
    Parameters:
        low: lowest reported face value tier
        high: highest reported face value tier
        avg: reported average price
        n: number of samples
        seed: random seed for reproducibility
    """
    rng = np.random.default_rng(seed)
    
    # Use the midpoint of low/high as the log-mean anchor (more stable than avg
    # which can be skewed by resale data in recent years)
    midpoint = (low + high) / 2
    log_mean = np.log(max(midpoint, 1))
    if high > low and high > 0:
        log_std = (np.log(max(high, 1)) - np.log(max(low, 1))) / 4
    else:
        log_std = 0.3  # Default modest spread for single-tier years
    
    log_std = max(log_std, 0.05)  # Prevent zero-width distributions
    
    samples = rng.lognormal(mean=log_mean, sigma=log_std, size=n)
    # Soft filter: discard extreme outliers instead of hard clipping
    # (hard clipping creates artificial spikes at boundaries)
    mask = (samples >= low * 0.5) & (samples <= high * 2.0)
    samples = samples[mask]
    return samples


selected_years = [1980, 1995, 2010, 2024]
selected_data = [df[df['year'] == yr].iloc[0] for yr in selected_years]
subplot_titles = [f"SB {row['sb_roman']} ({row['year']})" for row in selected_data]

fig = make_subplots(rows=2, cols=2, subplot_titles=subplot_titles,
                    horizontal_spacing=0.12, vertical_spacing=0.15)

hist_colors = ['#636EFA', '#EF553B', '#00CC96', '#AB63FA']

for i, (yr, row_data) in enumerate(zip(selected_years, selected_data)):
    samples = simulate_tier_distribution(
        row_data['face_value_low'], row_data['face_value_high'],
        row_data['avg_ticket_price'], seed=42 + i
    )
    r, c = divmod(i, 2)
    
    fig.add_trace(go.Histogram(
        x=samples, nbinsx=40,
        marker_color=hist_colors[i], opacity=0.75,
        name=f'{yr}', showlegend=True,
        hovertemplate='$%{x:,.0f}<br>Count: %{y}<extra></extra>'
    ), row=r+1, col=c+1)
    
    # Vertical line at reported average
    fig.add_vline(
        x=row_data['avg_ticket_price'],
        line_dash="dash", line_color="black", line_width=1.5,
        annotation_text=f"Avg: ${row_data['avg_ticket_price']:,.0f}",
        annotation_font_size=10,
        row=r+1, col=c+1
    )
    
    fig.update_xaxes(tickformat='$,.0f', row=r+1, col=c+1)

fig.update_layout(
    title=dict(
        text='Estimated Ticket Price Distributions (Simulated from Reported Ranges)',
        font_size=16
    ),
    height=600, width=1000,
    showlegend=False,
)

# Caveat footnote
fig.add_annotation(
    text="Note: Distributions are simulated estimates based on reported low/avg/high values, not observed transactions.",
    xref="paper", yref="paper", x=0.5, y=-0.08,
    showarrow=False, font=dict(size=10, color="gray")
)

fig.show()
fig.write_image(EXPORT_DIR / "histogram_selected_years.png", scale=2)

<ipython-input-7-6813f1a570bf>:84: DeprecationWarning:


Support for Kaleido versions less than 1.0.0 is deprecated and will be removed after September 2025.
Please upgrade Kaleido to version 1.0.0 or greater (`pip install 'kaleido>=1.0.0'` or `pip install 'plotly[kaleido]'`).




### What to Watch For: Simulated Data

- **The 1980 histogram is narrow** because there was only one price tier ($30). The simulation reflects this — almost all simulated tickets cluster tightly around $30.
- **The 2024 histogram is wide** because the face value range was $950–$9,400. This spread is real and reflects modern multi-tier pricing.
- **Never present simulations as observed data.** The footnote is not optional — it's the difference between honest analysis and misleading visualization.

> **Tip:** If you must simulate, use a fixed random seed (we use `seed=42`) so anyone running the notebook gets the same results. Reproducibility is non-negotiable.

---
## Chart 3b: Distribution of All Inflation-Adjusted Averages

In [8]:
fig = go.Figure()

fig.add_trace(go.Histogram(
    x=df['avg_ticket_price_adj'],
    nbinsx=25,
    marker_color=COLORS['histogram'],
    opacity=0.8,
    name='Inflation-Adjusted Avg',
    hovertemplate='$%{x:,.0f}<br>Count: %{y}<extra></extra>'
))

median_val = df['avg_ticket_price_adj'].median()
mean_val = df['avg_ticket_price_adj'].mean()

fig.add_vline(x=median_val, line_dash="dash", line_color="black",
              annotation_text=f"Median: {median_val:,.0f}",
              annotation_font_size=11)
fig.add_vline(x=mean_val, line_dash="dot", line_color=COLORS['adjusted'],
              annotation_text=f"Mean: {mean_val:,.0f}",
              annotation_font_size=11, annotation_position="top left")

fig.update_layout(
    title=dict(
        text=f'Distribution of Inflation-Adjusted Average Ticket Prices (All 60 Super Bowls)',
        font_size=16
    ),
    xaxis_title=f'Average Ticket Price ({TARGET_YEAR} Dollars)',
    yaxis_title='Number of Super Bowls',
    xaxis_tickformat='$,.0f',
    width=900, height=450,
)

fig.show()
fig.write_image(EXPORT_DIR / "histogram_all_years_pooled.png", scale=2)

<ipython-input-8-d7e77b711d0e>:34: DeprecationWarning:


Support for Kaleido versions less than 1.0.0 is deprecated and will be removed after September 2025.
Please upgrade Kaleido to version 1.0.0 or greater (`pip install 'kaleido>=1.0.0'` or `pip install 'plotly[kaleido]'`).




> **What this shows:** The distribution is heavily right-skewed. Most Super Bowls in history had inflation-adjusted average prices clustered under a few thousand dollars. The 2020s are dramatic outliers — a structural shift, not just inflation.

> **Tip:** When mean and median diverge significantly, the distribution is skewed. Always report both. The median is often a better "typical" value for skewed data.

---
## Chart 4: Ticket Price Growth vs. Inflation Rate

In [9]:
plot_df = df.iloc[1:].copy()  # Skip first row (no YoY for SB I)

fig = go.Figure()

fig.add_trace(go.Bar(
    x=plot_df['year'], y=plot_df['yoy_pct_change'],
    name='Ticket Price YoY Change',
    marker_color=COLORS['nominal'],
    opacity=0.7,
    hovertemplate='%{x}<br>Ticket: %{y:+.1f}%<extra></extra>'
))

fig.add_trace(go.Scatter(
    x=plot_df['year'], y=plot_df['cpi_yoy_pct_change'],
    name='CPI YoY Change',
    mode='lines',
    line=dict(color=COLORS['cpi'], width=2.5),
    hovertemplate='%{x}<br>CPI: %{y:+.1f}%<extra></extra>'
))

fig.add_hline(y=0, line_dash="solid", line_color="gray", line_width=0.5)

fig.update_layout(
    title=dict(text='Ticket Price Growth vs. General Inflation', font_size=18),
    xaxis_title='Year',
    yaxis_title='Year-over-Year Change (%)',
    yaxis_ticksuffix='%',
    width=1000, height=500,
    legend=dict(x=0.02, y=0.98),
    hovermode='x unified',
)

fig.show()
fig.write_image(EXPORT_DIR / "yoy_growth_vs_cpi.png", scale=2)

<ipython-input-9-1418bea4402c>:34: DeprecationWarning:


Support for Kaleido versions less than 1.0.0 is deprecated and will be removed after September 2025.
Please upgrade Kaleido to version 1.0.0 or greater (`pip install 'kaleido>=1.0.0'` or `pip install 'plotly[kaleido]'`).




> **What this shows:** In most years, ticket price increases dwarf the general inflation rate. The spikes (SB XXX in 1996, SB LV in 2021) represent structural changes in NFL pricing strategy, not gradual market forces.

> **Tip:** Overlaying a bar chart (discrete events) with a line chart (continuous trend) is an effective way to show "how does X compare to Y over time" without resorting to dual axes.

---
## Chart 5: Era Comparison

In [10]:
era_stats = df.groupby('era', observed=True).agg(
    avg_nominal=('avg_ticket_price', 'mean'),
    avg_adjusted=('avg_ticket_price_adj', 'mean'),
    min_adjusted=('avg_ticket_price_adj', 'min'),
    max_adjusted=('avg_ticket_price_adj', 'max'),
    count=('year', 'count')
).reset_index()

fig = go.Figure()

fig.add_trace(go.Bar(
    x=era_stats['era'].astype(str), y=era_stats['avg_nominal'],
    name='Avg Nominal Price',
    marker_color=COLORS['nominal'],
    text=era_stats['avg_nominal'].apply(lambda x: f'${x:,.0f}'),
    textposition='outside',
    hovertemplate='%{x}<br>Nominal: $%{y:,.0f}<extra></extra>'
))

fig.add_trace(go.Bar(
    x=era_stats['era'].astype(str), y=era_stats['avg_adjusted'],
    name=f'Avg in {TARGET_YEAR} Dollars',
    marker_color=COLORS['adjusted'],
    text=era_stats['avg_adjusted'].apply(lambda x: f'${x:,.0f}'),
    textposition='outside',
    hovertemplate='%{x}<br>Adjusted: $%{y:,.0f}<extra></extra>'
))

fig.update_layout(
    title=dict(text='Super Bowl Ticket Prices by Era', font_size=18),
    barmode='group',
    xaxis_title='Era',
    yaxis_title='Average Price (USD)',
    yaxis_tickformat='$,.0f',
    width=900, height=500,
    bargap=0.2,
    bargroupgap=0.1,
)

fig.show()
fig.write_image(EXPORT_DIR / "era_comparison.png", scale=2)

<ipython-input-10-6c4d520cd576>:41: DeprecationWarning:


Support for Kaleido versions less than 1.0.0 is deprecated and will be removed after September 2025.
Please upgrade Kaleido to version 1.0.0 or greater (`pip install 'kaleido>=1.0.0'` or `pip install 'plotly[kaleido]'`).




> **What this shows:** Even after adjusting for inflation, the 2020-2026 era is in a league of its own. The gap between nominal and adjusted bars narrows as you approach the present — that's just how inflation adjustment works (recent dollars need less adjustment).

> **Tip:** Grouped bar charts work well for 4-8 categories. Beyond that, they get cluttered. If you have more than ~8 groups, consider a heatmap or small multiples instead.

---
## Summary Statistics

In [11]:
summary = df[['year', 'sb_roman', 'city', 'avg_ticket_price', 'avg_ticket_price_adj',
              'resale_avg', 'winner']].copy()
summary.columns = ['Year', 'SB', 'City', 'Avg Price', f'In {TARGET_YEAR}$',
                    'Resale Avg', 'Winner']

summary.style.format({
    'Avg Price': '${:,.0f}',
    f'In {TARGET_YEAR}$': '${:,.0f}',
    'Resale Avg': lambda x: '${:,.0f}'.format(x) if pd.notna(x) else '—'
}).set_caption("Complete Super Bowl Ticket Price History")

,Year,SB,City,Avg Price,In 2025$,Resale Avg,Winner
0,1967,I,Los Angeles,$12,$116,—,Green Bay Packers
1,1968,II,Miami,$12,$111,—,Green Bay Packers
2,1969,III,Miami,$12,$105,—,New York Jets
3,1970,IV,New Orleans,$15,$124,—,Kansas City Chiefs
4,1971,V,Miami,$15,$119,—,Baltimore Colts
5,1972,VI,New Orleans,$15,$116,—,Dallas Cowboys
6,1973,VII,Los Angeles,$15,$109,—,Miami Dolphins
7,1974,VIII,Houston,$15,$98,—,Miami Dolphins
8,1975,IX,New Orleans,$20,$120,—,Pittsburgh Steelers
9,1976,X,Miami,$20,$113,—,Pittsburgh Steelers


---
## Key Takeaways

1. **$12 then ≈ $116 now.** Even adjusted for inflation, the first Super Bowl ticket was a bargain. Every era since has seen real price increases beyond what inflation alone would explain.

2. **The resale market is the real story.** Since secondary platforms became dominant (~2007), the gap between face value and what fans actually pay has grown from ~3x to ~5x. The NFL captures face value; the market captures everything above it.

3. **The 2020s are an outlier era.** COVID scarcity, destination cities (Las Vegas, Los Angeles), and corporate demand have pushed prices into unprecedented territory.

4. **Distribution matters.** A single "average price" hides enormous variation. The cheapest and most expensive seats at a modern Super Bowl can differ by 10x.

---

## Tips for Publishing on Medium

1. **Lead with the most surprising number.** "That $12 ticket in 1967 would cost $116 today — still cheaper than any Super Bowl since 1990."
2. **One chart per section.** Medium readers scan. Don't stack charts without narrative between them.
3. **Caption every image** with a one-sentence takeaway, not just a title.
4. **Static PNGs at 2x scale** (used throughout this notebook) ensure crisp rendering on Retina displays.
5. **Always provide alt-text** when uploading images to Medium for accessibility.
6. **Link to this notebook** for readers who want to explore the data or methodology further.
7. **Disclose your methodology** — readers who care will check, and those who don't will still trust you more for being transparent.

In [12]:
# Verify all exports
expected = [
    "trend_nominal_vs_adjusted.png",
    "face_vs_resale.png",
    "histogram_selected_years.png",
    "histogram_all_years_pooled.png",
    "yoy_growth_vs_cpi.png",
    "era_comparison.png",
]
found = [f.name for f in EXPORT_DIR.glob("*.png")]
for name in expected:
    status = "OK" if name in found else "MISSING"
    print(f"  {status}: {name}")

print(f"\nTotal PNGs: {len(found)} / {len(expected)} expected")

  OK: trend_nominal_vs_adjusted.png
  OK: face_vs_resale.png
  OK: histogram_selected_years.png
  OK: histogram_all_years_pooled.png
  OK: yoy_growth_vs_cpi.png
  OK: era_comparison.png

Total PNGs: 6 / 6 expected
